![](https://learnopencv.com/wp-content/uploads/2022/03/MediaPipe-pose-BlazePose-Topology.jpg)

In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # FIRST
    b = np.array(b) # MID
    c = np.array(c) # LAST
    
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle = 360-angle
    
    return angle
    

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [9]:
counter = 0
stage = None


with mp_pose.Pose(static_image_mode=False,
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    
    cap = cv2.VideoCapture(0)
    # For "mac" the destroyAllWindows does not work so using "startWindowThread"
#     cv2.startWindowThread()
    
    while cap.isOpened():
        ret,frame = cap.read()
        
        if not ret:
            print("Error occured while loading the webcam")
            
        # BGR -> RGB
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        img.flags.writeable=False
        
        results = pose.process(img)

        img.flags.writeable = True

        # RGB -> BGR
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        
        try:
            # landmarks
            landmarks = results.pose_landmarks.landmark

            # shoulder       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            left_shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]                                     
            right_sholder = [ landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Elobow = 
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]                                     
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            # Wrist
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]                                     
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            left_angle = np.round(calculate_angle(left_shoulder,left_elbow,left_wrist),2)
            right_angle  = np.round(calculate_angle(right_sholder,right_elbow,right_wrist),2)
        
            # Displaying the angle
            if landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility > 0.5:
                cv2.putText(img, str(left_angle), tuple(np.multiply(left_elbow, [1040, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, 
                            (255, 255, 255), 2, cv2.LINE_AA)

            # Displaying the angle
            if landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility > 0.5:
                cv2.putText(img, str(right_angle), tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, 
                            (255, 255, 255), 2, cv2.LINE_AA)
            
            if (left_angle>150) and (right_angle>150):
                stage="Down"
            if ( (left_angle<30) and (right_angle<30) ) and stage=="Down":
                stage="UP"
                counter = counter + 1
                print("counter = ",counter)
            
        except:
            pass
        
        # Render curl counter
        # Setup status box 138, 68, 15
        cv2.rectangle(img, (0,0), (270,93), (15,68,138), -1)


        cv2.putText(img, 'REPS', (15,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.putText(img, str(counter), 
                    (10,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(img, 'STAGE', (125,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.putText(img, stage, 
                    (95,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        mp_drawing.draw_landmarks(img,
                                  results.pose_landmarks,
                                  mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(30,100,250),
                                                         thickness=2,
                                                         circle_radius=3),
                                  mp_drawing.DrawingSpec(color=(200,40,150),
                                                         thickness=2,
                                                         circle_radius=3))
        
        
        cv2.imshow("AI GYM", img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    
    # only for "macOS" as destroy window by default not works in my mac 
    # its just a trick
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

I0000 00:00:1707754647.115462       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
